## Download the data

In [1]:
import numpy as np
import os
import tensorflow as tf
import pandas as pd

if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]
###### Do not modify here ###### 


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


## Get the data shape

In [2]:
print('Train', mnist.train.num_examples, 
      'Validation', mnist.validation.num_examples,
      'Test', mnist.test.num_examples)

print('Train images :', X_train1.shape,
      'Labels :', y_train1.shape)

print('Validation images :', X_valid1.shape,
      'Labels :', y_valid1.shape)

print('Test images :', X_test1.shape,
      'Labels :', y_test1.shape)


Train 55000 Validation 5000 Test 10000
Train images : (28038, 784) Labels : (28038,)
Validation images : (2558, 784) Labels : (2558,)
Test images : (5139, 784) Labels : (5139,)


## One-hot encoded y values

In [3]:
y_train_one_hot = np.zeros((len(y_train1), 5))
y_train_one_hot[np.arange(len(y_train1)), y_train1] = 1

y_valid_one_hot = np.zeros((len(y_valid1), 5))
y_valid_one_hot[np.arange(len(y_valid1)), y_valid1] = 1

y_test_one_hot = np.zeros((len(y_test1), 5))
y_test_one_hot[np.arange(len(y_test1)), y_test1] = 1

print('Check some examples of one-hot encoding:')
for i in range(10):
    print(y_train1[i], y_train_one_hot[i], '\n')

print ("Train Label: ", y_train_one_hot.shape)
print ("Validation Label: ", y_valid_one_hot.shape)
print ("Test Label: ", y_test_one_hot.shape)


Check some examples of one-hot encoding:
3 [ 0.  0.  0.  1.  0.] 

4 [ 0.  0.  0.  0.  1.] 

1 [ 0.  1.  0.  0.  0.] 

1 [ 0.  1.  0.  0.  0.] 

0 [ 1.  0.  0.  0.  0.] 

0 [ 1.  0.  0.  0.  0.] 

3 [ 0.  0.  0.  1.  0.] 

1 [ 0.  1.  0.  0.  0.] 

2 [ 0.  0.  1.  0.  0.] 

0 [ 1.  0.  0.  0.  0.] 

Train Label:  (28038, 5)
Validation Label:  (2558, 5)
Test Label:  (5139, 5)


## Hyper parameters

In [4]:
# hyper parameters
lr = 0.005
batch_size = 256
epochs = 350
saturate_limit = 20  # for applying early stopping
fold_num = 10


## Model

In [5]:
inputs_num = 784
outputs_num = 5

he_init = tf.contrib.layers.variance_scaling_initializer()

def dnn(X):
    # hidden layers
    dense1 = tf.layers.dense(inputs=X, units=128, activation=tf.nn.elu, kernel_initializer=he_init, name='dense1')
    dense2 = tf.layers.dense(inputs=dense1, units=128, activation=tf.nn.elu, kernel_initializer=he_init, name='dense2')
    dense3 = tf.layers.dense(inputs=dense2, units=128, activation=tf.nn.elu, kernel_initializer=he_init, name='dense3')
    dense4 = tf.layers.dense(inputs=dense3, units=128, activation=tf.nn.elu, kernel_initializer=he_init, name='dense4')
    dense5 = tf.layers.dense(inputs=dense4, units=128, activation=tf.nn.elu, kernel_initializer=he_init, name='dense5')
    return dense5
        
X = tf.placeholder(tf.float32, shape=(None, inputs_num), name='X')  # input layer
y = tf.placeholder(tf.int64, shape=(None), name='y')  # label

dnn_outputs = dnn(X)

#import pdb; pdb.set_trace()
logits = tf.layers.dense(inputs=dnn_outputs, units=outputs_num, activation=None, kernel_initializer=he_init, name='logits')
Y_probability = tf.nn.softmax(logits, name='Y_probability')

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(cross_entropy, name='loss')

optimizer = tf.train.AdamOptimizer(lr)
training_op = optimizer.minimize(loss, name='training_op')

correct = tf.nn.in_top_k(logits, y, k=1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')


## Training

In [6]:
# import sklearn Kfold to implement cross-validation
from sklearn.model_selection import KFold

# parameters
saturate_count = 0
best_acc = 0.
best_epoch = -1

# Merge training and validation set for K-fold cross validation
X_data = np.concatenate((X_train1, X_valid1), axis=0)
y_data = np.concatenate((y_train_one_hot, y_valid_one_hot), axis=0)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # 10-folder
    kf = KFold(n_splits=fold_num)
    fold_flag = 1
    for train_index, test_index in kf.split(X_data):
        X_train_tmp, X_valid_tmp = X_data[train_index], X_data[test_index]
        y_train_tmp, y_valid_tmp = y_data[train_index], y_data[test_index]
        
        # training for number of epochs times
        for e in range(epochs):
            for i in range(int(X_train_tmp.shape[0] / batch_size)):
                if(i==0):
                    batch_x = X_train_tmp[:batch_size]
                    batch_y = y_train_tmp[:batch_size]
                else:
                    batch_x = X_train_tmp[i * batch_size : (i + 1) * batch_size]
                    batch_y = y_train_tmp[i * batch_size : (i + 1) * batch_size]
                    
                sess.run(training_op, feed_dict={X: batch_x, y: batch_y})
        
            # validate
            loss, acc = sess.run([loss_function, accuracy], feed_dict={X: X_valid_tmp, y: y_valid_tmp})
        
            if best_acc < acc:
                best_acc = acc
                best_epoch = e
                saturate_count = 0
            else:
                saturate_count += 1
                if saturate_count >= saturate_limit:  # stop if saturate
                    break
    
        # print the best epoch results
        print('*' * 60)
        print('fold:', fold_flag)
        print('Best epoch:', best_epoch)
        print('Best accurancy:', best_acc)
        print('*' * 60, '\n')
        fold_flag = fold_flag + 1
    
    # print the test data accurancy
    print('=' * 60)
    print("Test data accurancy", sess.run(accuracy,
                                          feed_dict={X: X_test1,
                                                     y: y_test_one_hot}))
    print('=' * 60, '\n')
    
    # get prediction results
    predict_result = sess.run(tf.argmax(logits, 1), 
                              feed_dict={X: X_test1})
    
    # count the true or false num of label
    recall_matrix = np.zeros((5, 2))
    precision_matrix = np.zeros((5, 2))
    compare_result = sess.run(tf.equal(predict_result, y_test1))
   
    # construct the confusion matrix
    for i in range(len(predict_result)):
        if(compare_result[i]==True):
            recall_matrix[y_test1[i]][0] += 1
            precision_matrix[predict_result[i]][0] += 1
        else:
            recall_matrix[y_test1[i]][1] += 1
            precision_matrix[predict_result[i]][1] += 1
            
    # count the recall and precision value for every label
    label_dict = []
    for i in range(5):
        tmp_obj = {}
        tmp_obj['label'] = i
        tmp_obj['Precision'] = i
        pre =  precision_matrix[i][0]/(precision_matrix[i][0] + precision_matrix[i][1])
        rec = recall_matrix[i][0]/(recall_matrix[i][0] + recall_matrix[i][1])
        tmp_obj['Precision'] = pre
        tmp_obj['Recall'] = rec
        label_dict.append(tmp_obj)
        
    label_df = pd.DataFrame(label_dict)
        
    

InvalidArgumentError: labels must be 1-D, but got shape [256,5]
	 [[Node: SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT64, _device="/job:localhost/replica:0/task:0/gpu:0"](logits/BiasAdd, _arg_y_0_1/_7)]]

Caused by op 'SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits', defined at:
  File "C:\Users\starknight\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\starknight\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\starknight\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\starknight\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\starknight\Anaconda3\envs\tensorflow\lib\

### Show precision and recall rates via pandas

In [ ]:
label_df[['label', 'Precision', 'Recall']]